# Napari

Napari is an interactive, multi-dimensional open-source image viewer based on Python. 

It’s designed for fast browsing, annotating, and analyzing large multi-dimensional images. 

You can perform basic image processing steps just like you would do in other software tools such as ImageJ/Fiji. 

Like the latter, Napari can be extended with plugins written by an active user community to perform more advanced image analysis tasks, such as deep learning-based image segmentation.

---
## Setting Up Napari in Jupyter Notebook

We'll learn how to control Napari programmatically from a Jupyter Notebook. This unlocks the ability to create reproducible analysis pipelines that still allow for interactive visual inspection.

**The Jupyter-Napari Bridge**

The core concept is simple:
- We import napari and create a viewer object in our notebook.
- This viewer object is a live, two-way connection to the Napari window.
- Any action we perform on the viewer object in our code (like adding an image) will instantly update the Napari window.

First we need to import Napari library

In [ ]:
# Import the napari package
import napari

# Import other libraries
import numpy as np

To open Napari from notebook, run the following cell

Note: When using Napari inside a Jupyter Notebook, certain GUI elements or functionalities might be limited compared to running it standalone. For example, the scripting editor (console) is disabled in Napari GUI.

In [ ]:
# Create a new viewer instance
viewer = napari.Viewer() 

Napari viewer is an object

Inspect its methods and functions by typing 'viewer.'

In [ ]:
viewer.layers # prints the currently existing layers

---
## Loading and Displaying Images


To display images, we often use specialized libraries for image Input/Output operations.

We will use **imread** function from **io** module of **scikit-image** library.

In [ ]:
from skimage.io import imread # import image reading function

image = imread(r'../data/nuc_image.tif') # read image specified by path

Image is currently stored inside variable in jupyter notebook

To open image in Napari, we use the **add_image** function

In [ ]:
# Display the image in Napari

viewer.add_image(image)

# Repeating this cell will result in generation of new layers

If not specified, the default settings are gray colormap and translucent_no_depth blending.

We can change these parameters.

In [ ]:
# Display the same image with adjusted parameters

viewer.add_image(image, name='nuclei', colormap='inferno', contrast_limits=(5000,30000), blending='additive')


Binary or label images in common format can be also read as image using scikit-image library

In [ ]:
# Reading annotations

label_image = imread(r'../data/nuc_labels.tif')

We can load label image directly into napari as napari 'label' layer by using **add_labels** function

This time we will also store the layer under a variable label_layer

In [ ]:
# Add labels to viewer

label_layer = viewer.add_labels(label_image) # asignment of viewer label layer to varible

When layer is asigned to a variable, we can andjust its properties directly from notebook

In [ ]:
# adjust label contous and opacity parameters

label_layer.contour = 2
label_layer.opacity = 0.8

---
## Layer manipulation

Napari layers can be loaded back to jupyter notebook and stored as variables

Let's first display list of all layers in napari viewer

In [ ]:
# Display current layers

viewer.layers

In [ ]:
# Display it prettier 

for layer in viewer.layers:
    print(layer) # prints only layer name

Layers from list can be accessed by index or name for further manipulation

In [ ]:
# Accessing layers by index or name

image_layer = viewer.layers[1]
print(image_layer) # prints name of image layer
print(type(image_layer))

image_layer = viewer.layers['nuclei']
image_layer # prints output of image layer


In [ ]:
# Retrieve image data from a layer
image_data = image_layer.data

image_data

In [ ]:
# Retrieve manualy adjusted mask example

# Let's create mask with threshold and send it to Napari
threshold = 10000
mask = image_data > threshold
viewer.add_labels(mask, name='threshold')

# Inside Napari GUI modify mask as need
# ...


In [ ]:
# Read mask back to Napari
modified_mask = viewer.layers['threshold']

import matplotlib.pyplot as plt
plt.imshow(modified_mask.data)

---

### Removing layers

Layers can also be removed from Napari viewer from within notebook

In [ ]:
# Remove the layer by name
viewer.layers.remove('nuclei')

# Remove a layer using a direct reference to the layer object
viewer.layers.remove(modified_mask)

# Remove the layer by index
viewer.layers.pop(-1)

In [ ]:
# Remove all layers from the viewer
viewer.layers.clear()

#### Multi-dimensional images

In [ ]:
# Load a multi-dimensional dataset 
image_stack = imread(r'../data/mitosis.tif')

print("Loaded a 5D image stack with the following properties:")
print(f"Shape: {image_stack.shape}")
print(f"Data type: {image_stack.dtype}")

Napari is designed to handle multi-dimensional data out of the box. Let's send our entire nD image_stack to the viewer. Notice how Napari automatically detects the extra dimensions and creates sliders for them at the bottom of the canvas.

In [ ]:
# Add the entire 5D array as a single image layer.
# The 'name' argument is very useful for keeping your layers organized.
viewer.add_image(image_stack, name='Stack')

While viewing the full stack is great for browsing, you often want to control the appearance of each channel independently (e.g., assign different colors, change contrast). The best way to do this is to slice the array in Python and add each channel as a separate layer.

In [ ]:
# First, slice the full stack to isolate each channel.
# We are taking all frames and z-slices, a specific channel, and whole height/width.
channel_0 = image_stack[:, :, 0, :, :] 
channel_1 = image_stack[:, :, 1, :, :] 


# Now, add each nD channel image as a separate image layer with a unique color.
# `colormap` sets the color, and `blending='additive'` makes them overlay correctly.
viewer.add_image(channel_0, name='DNA', colormap='green', blending='additive')
viewer.add_image(channel_1, name='Microtubules', colormap='magenta', blending='additive')

Switch to 3D rendering in Napari (Toggle 3D View button)

See that image data array looks flat - it has no calibration information

In [ ]:
# In order to display a 3D stack with correct depth, we must set the scale parameter

calibration = [1, 1., 0.0885, 0.0885] # must be same lenght and order as image dimensions

viewer.add_image(channel_0, scale = calibration, name='DNA', colormap='green', blending='additive')
viewer.add_image(channel_1, scale = calibration, name='Microtubules', colormap='magenta', blending='additive')

---
## Uploading other types of layers

To open image in Napari, we use the **add_image** function. 

We can load label/mask image directly into napari as napari 'label' layer by using **add_labels** function

In a similar way we can add points or shapes layers by using **add_shapes** or **add_points**

In [ ]:
# Adding shapes layer
# Switch to 2D view

polyline = [[10, 3],
            [12,5],
            [12,9],
            [10, 11]] # polyline defined by coordinates

shapes = viewer.add_shapes(polyline, face_color='white', edge_width=0, opacity=1)

In [ ]:
# Adding points layer
# Points are loaded from a 2-column array (y and x coordinates)

points_arr = [[7,5],
              [7,9]] # point coordinates

points = viewer.add_points(points_arr, border_color='black', face_color='red', size=2)

New objects can be added to stored layers

In [ ]:
# Adding points to existing layer

points.add([9,7])

---

### Viewer settings

You can programmatically control the viewer overlay 

Beyond just adding data, you can control nearly every aspect of the Napari viewer's state directly from your code. This is extremely powerful for setting up a standardized, reproducible view of your data without ever touching the GUI.

Let's explore some of the most useful viewer properties.

```python
# Enable/disable the grid view
viewer.grid.enabled = True
viewer.grid.enabled = False

# Set the viewer to display in 2D/3D rendering mode.
viewer.dims.ndisplay = 3
viewer.dims.ndisplay = 2
```

---
#### Making snapshots to document your steps

With the following command, we can make a screenshot of Napari and show it in our notebook.

In [ ]:
napari.utils.nbscreenshot(viewer)

In [ ]:
napari.utils.nbscreenshot(viewer, canvas_only = True) # only canvas, no menus

---

##### --- Exercise ---

1. **Load the Image:**  
   Read a 2D image file from path into memory using `skimage.io.imread()`.

2. **Segment the Objects:**  
   Filter image to remove noise.
   Apply an appropriate thresholding method (e.g., Otsu’s method) to separate foreground objects from the background.  
   Then, label the detected objects using `skimage.measure.label()`.

3. **Compute Object Centroids:**  
   Use `skimage.measure.regionprops_table()` to measure object properties and extract the centroid coordinates - select properties ('label', 'area', 'centroid')
    - *Note:* regionprops gives centroid-0 and centroid-1 for the y and x coordinates
    - Convert measurements dictionary into a pandas DataFrame

4. **Visualize in Napari:**  
   * Open a new Napari viewer instance directly from the notebook.  
   * Add the image as the **image layer**.  
   * Add the labeled mask as the **labels layer**.  
   * Add the centroid coordinates as the **points layer**.
   - *Hint:* you can access values as array from DataFrame with `DataFrame[['col1', 'col2']].values`

5. **Create a Screenshot:**  
   Take a screenshot of your Napari viewer within the notebook using the built-in screenshot function (napari.utils.nbscreenshot()   

In [ ]:
image_path = '../data/blobs.tif'

# Your code here


<details>
<summary>Click to see the example solution</summary>

```python
# EXAMPLE SOLUTION

import numpy as np
import pandas as pd
import napari
from skimage import data, filters, morphology, measure, io

# Load and process one image
image = io.imread(image_path)
blurred_image = filters.gaussian(image, sigma=2.0)
otsu_threshold = filters.threshold_otsu(blurred_image)
mask = blurred_image > otsu_threshold
#cleaned_mask = morphology.binary_closing(mask)
label_image = measure.label(mask)

# Get measurements
properties = ('label', 'area', 'centroid')
props_df = pd.DataFrame(measure.regionprops_table(label_image, intensity_image=image, properties=properties))

# Optional - information printing
print("Data is ready. We have:")
print(f"- An image with shape {image.shape}")
print(f"- A label image with {label_image.max()} objects")
print("- A Pandas DataFrame with measurements:")
print(props_df.head())

# Create point coordinates
points_coordinates = props_df[['centroid-0', 'centroid-1']].values

# Optionaly rename columns in DataFrame first
# props_df.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)
# points_coordinates = props_df[['y', 'x']].values

# Visualize in Napari
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(label_image)
# Now, add the points to the viewer
viewer.add_points(points_coordinates, name='Centroids', size=5, face_color='cyan')

# Take a screenshot
napari.utils.nbscreenshot(viewer, canvas_only = True) # only canvas, no menus
```

**Bonus**

Let's make the size of each point proportional to the area of the object it represents.

Change the `props_df` and `points_coordinates` variable names to match your variables.

In [ ]:
# We can pass a list or array of values to the `size` argument.
# It will map each value to the size of the corresponding point.
point_sizes = props_df['area'].values / 50 # Divide by 50 to make the sizes reasonable

# We can also color each point
# We'll use a list of random values
n_points = len(props_df)
colors = np.random.rand(n_points, 4)  # random RGBA values
colors[:, 3] = 0.8  # set uniform opacity if desired

# Add a new points layer with these properties
viewer.add_points(
    points_coordinates,
    name='Customized Centroids',
    size=point_sizes,
    face_color=colors,
    opacity=0.7
)

**Bonus 2**

✨🧙 Custom widgets 🧙✨


This example demonstrates how to create an interactive thresholding tool in napari using magicgui.
The @magicgui decorator automatically turns a Python function into a small graphical user interface (GUI) — in this case, a slider widget that lets you adjust the threshold value dynamically.

In [ ]:
import napari
from skimage import measure, io
from magicgui import magicgui

# Load image
image = io.imread(image_path)

viewer = napari.Viewer()
viewer.add_image(image, name='Raw Image')

# Minimal threshold slider
@magicgui(threshold={'widget_type': 'FloatSlider', 'min': 0, 'max': image.max(), 'step': 1})
def apply_threshold(threshold):
    mask = (image > threshold)
    labels = measure.label(mask)

    # Update or create mask layer
    if 'Thr' in viewer.layers:
        viewer.layers['Thr'].data = mask
    else:
        viewer.add_labels(mask, name='Thr')
    
    # Update or create label layer
    if 'Thr_Labels' in viewer.layers:
        viewer.layers['Thr_Labels'].data = labels
    else:
        viewer.add_labels(labels, name='Thr_Labels')

viewer.window.add_dock_widget(apply_threshold)
